## Create tables for pension funds and their divisions

Create a database for pension funds. Start with creating a table for the funds themselves. The funds table has fundid, fundname, shortname and a regex column, representing a regular expression that can be used to determine if a string corresponds to that particular fund.

The second table is the subfund, or department or division of a fund. It has the columns subfundid, fundid, subfundname, shortname, subfundtype (shared or private), and regex.

Start by collecting the values from the latest excel file from FME. The funds are listed conveniently there.

In [ ]:
#Gera funds töflu
using XLSX, DataFrames

path = "./skjölin frá Birgi/Files/Arsreikningabok-2021.xlsx"

xf = XLSX.readxlsx(path)

#Return the values in a column, starting in first row and until first empty cell occurs
function read_column(sheet, firstrow, col)
    res = []
    row = firstrow
    while !ismissing(sheet[row,col])
        push!(res,sheet[row,col])
        row+=1
    end
    return res
end

sheet_name = "Gögn"

sheet = xf[sheet_name]
firstrow = 2
col = 2
shortname = read_column(sheet, firstrow, col)
col = 3
fundname = read_column(sheet, firstrow, col)
col = 4
subfundname = read_column(sheet, firstrow, col)
col = 5
subfundtype = read_column(sheet, firstrow, col)
col = 1
fullname = read_column(sheet, firstrow, col)
rawdata = DataFrame(fundname = fundname, shortname = shortname,
        subfundname = subfundname,subfundtype = subfundtype, fullname = fullname)

In [ ]:
using LibPQ, DataFrames

file = open("login_for_sql.txt", "r")
# Read the data from the file into a matrix
username = readline(file)
pass = readline(file)
# Close the file
close(file)

conn = LibPQ.Connection("dbname=Symmetria user=$username password=$pass")

result = execute(conn, "SELECT * FROM fund") |> DataFrame

execute(conn, "BEGIN;")

LibPQ.load!(
    (fund_name = funds.fundname, short_name = funds.shortname,regex_keyword=funds.regexp,
        founded = fill(missing,nrow(funds))),
    conn,
    "INSERT INTO fund (fund_name, short_name, regex_keyword, founded) VALUES (\$1, \$2,\$3,\$4);",
)

execute(conn, "COMMIT;")